# Sistemin Çalışma Mantığı
Başvuru yapan her bir adayın işe seçilmek için belirli bir olasılığı vardır. Her bir aday için birbirinden bağımsız olarak seçilme olasılığı tahmin edilecektir. En yüksek olasılığa sahip 5 aday işverene önerilecektir. <br>
Burada adayların birbirini tanımadığı için birbirinden etkilenmeyeceği düşünülerek adaylar arasında bir bağ olmadığı kabul edilmiştir.<br>
Her bir adayın özgeçmişi ile iş metni arasında farklı ilişkiler kurularak sayısal girdiler oluşturulmuş, bu girdiler ile ANN eğitilerek tahminleme sağlanmıştır.<br>
Bu ilişkiler aşağıda maddeler halinde açıklanmıştır.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Csv

In [ ]:
# INPUT
# Candidate Features
aday = pd.read_csv("../input/c/c/c/datathon-who-gets-the-job/Datathon_Aday.csv") # Aday ile ilgili okul bilgileri ve yaşadığı şehri
tecrube = pd.read_csv("../input/c/c/c/datathon-who-gets-the-job/Datathon_Tecrube.csv") # Adayların geçmiş iş tecrübesini
yetenek = pd.read_csv("../input/c/c/c/datathon-who-gets-the-job/Datathon_Yetenek.csv") # Adayın Kariyer.net üzerinden girmiş olduğu yetenekleri göstermektedir.

# Advertisement Features
ilan = pd.read_csv("../input/c/c/c/datathon-who-gets-the-job/Datathon_ilan.csv") # İlan metni, pozisyon adı ve lokasyon gibi bilgileri,

# OUTPUT
basvuru_recruited = pd.read_csv("../input/c/c/c/datathon-who-gets-the-job/Datathon_Basvuru_iseAlinanlar.csv") # Başvurusu sonucu işe alınan adayları

# PREDICT
basvuru = pd.read_csv("../input/c/c/c/datathon-who-gets-the-job/Datathon_Basvuru.csv") # Hangi adayın hangi iş ilanına başvurduğu
test = pd.read_csv("../input/c/c/c/datathon-who-gets-the-job/test.csv")


# Haversine Function
Bu fonksiyon koordinatlar arası mesafe ölçümü için kullanılmaktadır. Bir medium yazısından alınmıştır.<br>
Burada şehirlerin koordinat listeleri internetten farklı kaynaklardan toplanıp bir dataset oluşturulmuş bu dataset paylaşılmıştır.

In [ ]:
# Haversine Length Function
# https://medium.com/@sddkal/koordinatlar-arası-uzaklık-hesabı-için-haversine-fonksiyonu-982d90c550bf

import numpy as np
R = 6371
def pol2cart(lat, long):
    lat, long = np.radians(lat), np.radians(long)
    return R*np.cos(lat) *np.cos(long),\
           R*np.cos(lat) *np.sin(long),\
           R*np.sin(lat)


def haversine_dist(point1, point2):
  point1_cart = np.array(pol2cart(*point1))
  point2_cart = np.array(pol2cart(*point2))
  euc_dist = np.linalg.norm(point1_cart-point2_cart)
  sin_theta_2 = euc_dist / (R * 2)
  theta_2 = np.arcsin(sin_theta_2)
  theta = 2*theta_2
  dist = R*theta
  return dist

# Şehirlerin koordinat listesi import edilir. Bu liste internetten farklı websitelerinden derlenerek dataset olarak sunulmuştur.
coordinates = pd.read_csv("../input/turkeycoordinates/Coordinates.csv")

def city_dist(city1, city2):
    city1 = city1.replace("i", "İ").upper()
    city2 = city2.replace("i", "İ").upper()
    
    # İstanbul(Avr.) or İstanbul(Asya) -> "İSTANBUL"
    city1 = "İSTANBUL" if city1=='İSTANBUL(AVR.)' or city1=='İSTANBUL(ASYA)' else city1
    city2 = "İSTANBUL" if city2=='İSTANBUL(AVR.)' or city2=='İSTANBUL(ASYA)' else city2
    
    # Get coordinates
    city1 = coordinates[coordinates.City == city1]
    city2 = coordinates[coordinates.City == city2]
    
    # Eger sehirlerden birisi kayitli degilse 0 dondur
    if city1.lat.dtype != 'float64' or city2.lat.dtype != 'float64':
        return 0
    else:
        city1_coor = (city1.lat, city1.long)
        city2_coor = (city2.lat, city2.long)
        return haversine_dist(city1_coor, city2_coor)
    

# Review Data

In [ ]:
# aday.info()
# tecrube.info()
# yetenek.info()

# ilan.info()

# basvuru_recruited.info()

# basvuru.info()
#len(basvuru_recruited.AdayId.unique())

# Data'dan Sayısal İlişkiler Elde Etmek

## Distance
Adayın lokasyonunun ilanın lokasyonuna olan uzaklığı

In [ ]:
def distance(adayid, ilanid):
    # city1 = coordinates[coordinates.City == city1]
    try:
        aday_city = aday[aday.AdayId == adayid].SehirAdi.dropna()
        aday_city = aday_city.tolist()[0] # Eğer birden fazla tanımlanmışsa ilkini al
        ilan_city = ilan[ilan.IlanId == ilanid].lokasyon.dropna().tolist()[0]
    except IndexError:
        return 0
    return city_dist(aday_city, ilan_city)

distance(11178450, 2752333)

## Month of Experience - Position
Aday ilanda belirtilen pozisyonda daha önceden kaç ay çalışmış?

In [ ]:
def experience_position(adayid, ilanid):
    ilanExperience = ilan[ilan.IlanId == ilanid].pozisyonAdi.tolist()[0]
    adayExperiences = tecrube[tecrube.AdayId == adayid] # Adayın deneyimleri
    adayExperiences = adayExperiences[adayExperiences.PozisyonAdi == ilanExperience].CalismaAyi
    return adayExperiences.sum()

experience_position(16377536, 2077585)
# aday[aday.AdayId == 22742218].BitisYili

## Month of Experience - Industry
Aday ilanda belirtilen sektörde daha önceden kaç ay çalışmış?

In [ ]:
def experience_industry(adayid, ilanid):
    ilanExperience = ilan[ilan.IlanId == ilanid].sektorAdi.tolist()[0]
    adayExperiences = tecrube[tecrube.AdayId == adayid] # Adayın deneyimleri
    adayExperiences = adayExperiences[adayExperiences.SektorAdi == ilanExperience].CalismaAyi
    return adayExperiences.sum()

experience_industry(16377536, 2077585)

## Education Level
**Eğitim durumu.** <br>
0 -> Belirtilmemiş<br>
1 -> Önlisans<br>
2 -> Lisans<br>
3 -> Y.Lisans<br>
4 -> Doktora

In [ ]:
def eduLevel(adayid):
    aday_level = aday[aday.AdayId == adayid].Tip.tolist()[0]
    if aday_level == 'Belirtilmemiş':
        ret = 0
    elif aday_level == 'Önlisans':
        ret = 1
    elif aday_level == 'Lisans':
        ret = 2
    elif aday_level == 'Y.Lisans':
        ret = 3
    elif aday_level == 'Doktora':
        ret = 4
    else:
        ret = 0
    return ret

eduLevel(16377536)

## Year of Graduation
Adayın mezun olalı kaç sene olmuş?<br>
nan veya 0 için değer 0 yıl alınır.

In [ ]:
def yearGraduation(adayid):
    graduation_date = aday[aday.AdayId == adayid].BitisYili.tolist()[0]
    if np.isnan(graduation_date) or graduation_date == 0.0:
        ret = 0.0
    else:
        ret = 2021 - graduation_date
    return ret

yearGraduation(16377536)

## School
Okul ID'si direkt olarak alınır.

In [ ]:
def school(adayid):
    return aday[aday.AdayId == adayid].OkulId.tolist()[0]

school(16377536)

## Department Match
Adayın okuduğu bölümün ismi ilanda aranan niteliklerde geçiyor mu?

In [ ]:
def department_match(adayid, ilanid):
    try:
        aday_department = aday[aday.AdayId == adayid].BolumAdi.dropna().tolist()[0]
        ilan_qualifications = ilan[ilan.IlanId == ilanid].Nitelikler.tolist()[0].split()
    except IndexError:
        return 0
    if False in [i in ilan_qualifications for i in aday_department.split()]: # Eğer bölüm ismi (bir kaç kelime varsa herhangi biri) niteliklerde geçmiyorsa 0 döndür. Geçiyorsa 1 döndür.
        return 0
    else:
        return 1

department_match(2438572, 2631636)

## Ability
Adayın yetenekleri ilandaki kelimelerin ne kadarlık bir kısmında geçiyor? (%)

In [ ]:
def ability(adayid, ilanid):
    aday_abilities = yetenek[yetenek.AdayId == adayid].Yetenek.tolist()
    ilan_qualifications = ilan[ilan.IlanId == ilanid].Nitelikler.tolist()[0].split()
    total_mention = [ilan_qualifications.count(ability) for ability in aday_abilities]
    return sum(total_mention) / len(ilan_qualifications)

ability(16377536, 2077585)

## Job Text
İlan metninde geçen *tekrar etmeyen* kelimelerden ne kadarı adayın geçmiş deneyimlerinin açıklamalarında bulunuyor? (%)

In [ ]:
def job_text(adayid, ilanid):
    ilan_text = ilan[ilan.IlanId == ilanid].ilanMetni.tolist()[0].split()
    aday_text = tecrube[tecrube.AdayId == adayid].Aciklama.dropna().tolist() # Adayın deneyimlerinin text'leri
    aday_text = set([j for i in aday_text for j in i.split()]) # Adayın tüm deneyimlerinin tekrar etmeyen kelimeleri
    num = 0
    for i in ilan_text:
        if i in aday_text:
            num += 1
    return num/len(ilan_text)

job_text(16377536, 2077585)

## Qualifications Text
İlan niteliklerinde geçen kelimelerden ne kadarı adayın yeteneklerinin içerisinde? (%)

In [ ]:
def qualifications_text(adayid, ilanid):
    try:
        ilan_text = ilan[ilan.IlanId == ilanid].Nitelikler.dropna().tolist()[0].split()
        aday_text = list(yetenek[yetenek.AdayId == adayid].Yetenek.dropna().tolist()[0].split()) # Adayın yeteneği (split birden fazla kelime için kondu. En son bölünecek)
    except IndexError:
        return 0
    num = 0
    for i in ilan_text:
        if i in aday_text:
            num += 1
    return num/(len(ilan_text) * len(aday_text))

qualifications_text(2510453, 2131294)

# Sayısal Değerlerin Bulunduğu Data Listesinin Üretilmesi

In [ ]:
x_data = pd.DataFrame(columns=[
    "AdayId", 
    "IlanId", 
    "distance", 
    "experience_position", 
    "experience_industry", 
    "eduLevel", 
    "yearGraduation", 
    "school", 
    "department_match", 
    "ability", 
    "job_text", 
    "qualifications_text"
                              ])
x_data

In [ ]:
def add_data(data, adayid, ilanid, recruited = None):
    
    val = {
        "AdayId": adayid,
        "IlanId": ilanid,
        "distance": distance(adayid, ilanid),
        "experience_position": experience_position(adayid, ilanid),
        "experience_industry": experience_industry(adayid, ilanid),
        "eduLevel": eduLevel(adayid),
        "yearGraduation": yearGraduation(adayid),
        "school": school(adayid),
        "department_match": department_match(adayid, ilanid),
        "ability": ability(adayid, ilanid),
        "job_text": job_text(adayid, ilanid),
        "qualifications_text": qualifications_text(adayid, ilanid),
        "recruited": recruited
        }
    return data.append(val, ignore_index=True)

# x_data = add_data(x_data, 16377536, 2077585)
x_data

## Test Recruited or Not
Girilen adayın işe alınıp alınmadığını döndürür. 1 işe alındı, 0 alınmadı anlamına gelir.

In [ ]:
def ishired(adayid, ilanid):
    hired_persons = basvuru_recruited[basvuru_recruited.ilanId == ilanid].AdayId.tolist()
    if adayid in hired_persons:
        return 1
    else:
        return 0

## Generate Dataset 50k
2866 tanesi işe alınan adaylardan oluşan 50.000 adet veriden oluşan bir dataset üretir.<br>
Bu dataset 'dataset_50k.csv' ismiyle kaydedilir. Böylece train kısmından önce notebook kapatılıp sonradan devam edilebilir.

In [ ]:
# 50000 - 2866
dataset_50k = pd.concat([basvuru_recruited, basvuru[:(50000 - 2866)]], ignore_index=True)
dataset_50k

In [ ]:
import time
start = time.time()
for index,row in dataset_50k.iterrows():
    adayid, ilanid = row["AdayId"], row["ilanId"]
    if ishired(adayid, ilanid):
        x_data = add_data(x_data, adayid, ilanid, recruited=1)
    else:
        x_data = add_data(x_data, adayid, ilanid, recruited=0)

print(time.time() - start)

In [ ]:
x_data.to_csv("/kaggle/working/dataset_50k.csv")

## Generate Dataset Plus 100k
İsteğe bağlı olarak 100k kadar daha işe kabul edilmeyen kullanıcı verisi önceki dataset'in sonuna ilave edilebilir.

In [ ]:
# dataset_100kplus = basvuru[(50000 - 2866):(150000 - 2866)].reset_index()
# dataset_100kplus

In [ ]:
"""
import time
start = time.time()
for index,row in dataset_100kplus.iterrows():
    adayid, ilanid = row["AdayId"], row["ilanId"]
    if ishired(adayid, ilanid):
        x_data = add_data(x_data, adayid, ilanid, recruited=1)
    else:
        x_data = add_data(x_data, adayid, ilanid, recruited=0)

print(time.time() - start)
x_data.to_csv("/kaggle/working/dataset_150k.csv")
"""

# Train

## Import Tensorflow
Eğitim için TensorFlow framework'u kullanılmıştır.

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Import Dataset & Split Train/Test Data
Bu örnek için 50,000 veriye sahip bir dataset kullanılmış, train 45,000, test 5000 veri olacak şekilde ayrılmıştır.

In [ ]:
df = pd.read_csv("../input/kariyernet-datathlon/dataset_50k.csv")
df = df.fillna(0)
df

In [ ]:
inputs = df[["distance", "experience_position", "experience_industry", "eduLevel", "yearGraduation", "school", "department_match", "ability", "job_text", "qualifications_text"]]
outputs = df.recruited

x_train = inputs[:45000]
y_train = outputs[:45000]
x_test = inputs[45000:]
y_test = outputs[45000:]

## Define Model
Modelin çıktısı sigmoid fonksiyonuna sokulmuştur. Böylece 0 ile 1 arası bir değer elde edilmesi sağlanır.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

## Define Parameters
**Loss Function:** Binary Crossentropy<br>
**Optimizer:** Adam

In [ ]:
loss = keras.losses.BinaryCrossentropy()
optim = keras.optimizers.Adam(learning_rate=0.01)
metrics = ['accuracy']  # Burada saklamak istediğimiz bilgileri yazıyoruz.
model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [ ]:
batch_size = 64
epochs = 10

## Train

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

## Evaluate

In [ ]:
model.evaluate(x_test, y_test, batch_size=batch_size, verbose=2)

## Predict
Tahminleme aşamasında için her bir ilana başvuru yapan tüm adayların o ilanı kazanma olasılığı hesaplanır. En yüksek olasılığa sahip 5 aday işverene sunulur.

In [ ]:
predict_df = pd.DataFrame(columns=["ilanId", "AdayId"])

def add_predict(data, ilanid, adayid):
    val={
        "ilanId": ilanid,
        "AdayId": adayid
    }
    return data.append(val, ignore_index=True)

In [ ]:
# Make empty dataset
mk_dataset = lambda: pd.DataFrame(columns=[
    "AdayId", 
    "IlanId", 
    "distance", 
    "experience_position", 
    "experience_industry", 
    "eduLevel", 
    "yearGraduation", 
    "school", 
    "department_match", 
    "ability", 
    "job_text", 
    "qualifications_text"
                              ])

In [ ]:
# Add data
def id2data(dataset):
    x_data = mk_dataset()
    for index,row in dataset.iterrows():
        adayid, ilanid = row["AdayId"], row["ilanId"]
        
        x_data = add_data(x_data, adayid, ilanid)
    return x_data

In [ ]:
num = 1
for ilanid in test.ilanId:
    id_data = basvuru[basvuru.ilanId == ilanid].reset_index()
    r2p = id2data(id_data)
    r2p = r2p[["distance", "experience_position", "experience_industry", "eduLevel", "yearGraduation", "school", "department_match", "ability", "job_text", "qualifications_text"]]
    predict = model.predict(r2p)
    indices = (-predict.flatten()).argsort()[:5]
    id_list = id_data.AdayId.tolist()
    for indice in indices:
        adayid = id_list[indice]
        predict_df = add_predict(predict_df, ilanid, adayid)
    print(f"{num} / 776") # Sadece ilerleyişi görebilmek adına yazdırıldı.
    num += 1

## Tahminlerin İncelenmesi / Kaydedilmesi

In [ ]:
predict_df

In [ ]:
predict_df.to_csv("/kaggle/working/predict.csv")